<a href="https://colab.research.google.com/github/KESAVE8604-droid/kesaverdhen-hybrid-movie-recommender/blob/main/notebooks/Phase4_HybridFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Phase 4 — Hybrid Fusion Layer**

🌐 Goal

To combine the predictions of your Collaborative Filtering (CF) and Content-Based models into a single hybrid score for improved recommendation accuracy.

🧩 Approach 1: Weighted Average Fusion

We’ll implement the weighted hybrid equation:

HybridScore
=
𝛼
×
𝐶
𝐹
_
𝑆
𝑐
𝑜
𝑟
𝑒
+
(
1
−
𝛼
)
×
𝐶
𝑜
𝑛
𝑡
𝑒
𝑛
𝑡
_
𝑆
𝑐
𝑜
𝑟
𝑒
HybridScore=α×CF_Score+(1−α)×Content_Score

Where:

CF_Score → Predicted rating from the collaborative model

Content_Score → Similarity-based score from the TF-IDF model

α (alpha) → Tunable weight (default: 0.7

In [1]:
#Step 1 — Import Dependencies & Load Models
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
!pip install scikit-surprise --quiet


In [3]:
!pip install numpy==1.26.4 --force-reinstall --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
import joblib
import pandas as pd

# Load your saved models
tfidf = joblib.load("/content/drive/MyDrive/MovieRecommender/tfidf_vectorizer.pkl")
tfidf_matrix = joblib.load("/content/drive/MyDrive/MovieRecommender/tfidf_matrix.pkl")
cosine_sim = joblib.load("/content/drive/MyDrive/MovieRecommender/cosine_similarity.pkl")
cf_model = joblib.load("/content/drive/MyDrive/MovieRecommender/collab_model.pkl")

movies_df = pd.read_csv("/content/drive/MyDrive/MovieRecommender/movielens_cleaned.csv")

print("✅ All models and data loaded successfully!")


✅ All models and data loaded successfully!


In [5]:
#  Verify Model Functionality
from surprise import SVD

# Try a test prediction
prediction = cf_model.predict(1, 10)  # Predict rating for user 1, movie 10
print(f"Predicted rating for user 1 on movie 10: {prediction.est:.2f}")


Predicted rating for user 1 on movie 10: 3.65


In [6]:
#Step 2 — Define Helper Functions
#(a) Get Content-Based Similarity Scores
indices = pd.Series(movies_df.index, index=movies_df['Clean_Title'].str.lower()).drop_duplicates()

def get_content_score(movie_title, top_n=10):
    movie_title = movie_title.lower()
    if movie_title not in indices:
        print("❌ Movie not found in dataset.")
        return []

    idx = indices[movie_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:top_n+1]]

    return movies_df.iloc[top_indices][['MovieID', 'Clean_Title', 'Genres']]

#(b) Get Collaborative Filtering Predictions
def get_cf_score(user_id, movie_id):
    try:
        prediction = cf_model.predict(user_id, movie_id)
        return prediction.est
    except:
        return np.nan


In [7]:
#Step 3 — Hybrid Recommender Function
def hybrid_recommend(user_id, liked_movie, alpha=0.7, top_n=10):
    liked_movie = liked_movie.lower()

    if liked_movie not in indices:
        print("❌ Movie not found in content model.")
        return None

    # Step 1: Get top similar movies (content)
    similar_movies = get_content_score(liked_movie, top_n=50)
    if similar_movies.empty:
        print("❌ No similar movies found.")
        return None

    hybrid_scores = []
    base_idx = indices[liked_movie]

    # Step 2: Combine CF and content scores
    for _, row in similar_movies.iterrows():
        movie_id = row['MovieID']
        cf_score = get_cf_score(user_id, movie_id)
        content_score = cosine_sim[base_idx][indices[row['Clean_Title'].lower()]]

        # Weighted combination
        hybrid_score = alpha * cf_score + (1 - alpha) * content_score * 5  # normalize to 5-scale
        hybrid_scores.append((row['Clean_Title'], row['Genres'], hybrid_score))

    # Step 3: Sort final recommendations
    hybrid_df = pd.DataFrame(hybrid_scores, columns=['Movie', 'Genres', 'HybridScore'])
    hybrid_df = hybrid_df.sort_values(by='HybridScore', ascending=False).head(top_n).reset_index(drop=True)

    return hybrid_df


In [10]:
hybrid_results = hybrid_recommend(user_id=1, liked_movie="Toy Story", alpha=0.7, top_n=10)
display(hybrid_results)


NameError: name 'cf_model' is not defined

In [12]:
!pip install numpy==1.26.4
!pip install scikit-surprise --no-cache-dir --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 294.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 308.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 228.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 311.0 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611202 sha256=82972617f48424a561084ee06808cb8f44f2fcb44413f6a5ab2a9b7aa54f2ae6
  Stored in directory: /tmp/pip-ephem-wheel-cache-12z400wz/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise
  Attempting uninstall: numpy
    Foun

In [13]:
import joblib
import pandas as pd
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Load all models
print("🔄 Loading trained models...")
tfidf = joblib.load("/content/drive/MyDrive/MovieRecommender/tfidf_vectorizer.pkl")
tfidf_matrix = joblib.load("/content/drive/MyDrive/MovieRecommender/tfidf_matrix.pkl")
cosine_sim = joblib.load("/content/drive/MyDrive/MovieRecommender/cosine_similarity.pkl")
cf_model = joblib.load("/content/drive/MyDrive/MovieRecommender/collab_model.pkl")  # <== This defines cf_model ✅

# Load your cleaned movie dataset
movies_df = pd.read_csv("/content/drive/MyDrive/MovieRecommender/movielens_cleaned.csv")

# Get unique movie info
movies_only = movies_df[['MovieID', 'Clean_Title', 'Genres', 'Year']].drop_duplicates().reset_index(drop=True)

print("✅ All models and data loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 Loading trained models...


KeyboardInterrupt: 

In [11]:
print("movies_df shape:", movies_df.shape)
print("cosine_sim shape:", cosine_sim.shape)


movies_df shape: (1000209, 12)
cosine_sim shape: (3706, 3706)
